# This is a notebook on an article about blockchains from https://asthasr.github.io/posts/how-blockchains-work/

### Foundations
#### First, a hash algorithm is a way to convert a given string into an unpredictable string of a fixed length, called a digest.

In [12]:
from argparse import ArgumentParser
from hashlib import md5

def hash_string(string):
    hash = md5()
    hash.update(string.encode("utf-8"))
    return hash.hexdigest()

print(hash_string("ninja"))
print(hash_string("kongfu"))

3899dcbab79f92af727c2190bbd8abc5
03b1d09b804cbc0fd72a6d35109e5877


#### Hashes of this type are used to check passwords—you can check whether a password matches without storing the password itself. 
Blockchains are a kind of ledger: they have entries added to them over time. Hashes can help with that by protecting the ordering and contents of messages.

In [13]:
def hash_ledger_entry(string, previous_digest=None):
    """Hashes a string with the hash of previous entries in the ledger, if any."""
    hash = md5(string.encode("utf-8"))

    if previous_digest:
        hash.update(previous_digest.encode("utf-8"))

    return hash.hexdigest()


def generate_ledger(*strings):
    """Generates the entries in a ledger consisting of a set of strings."""
    digest = None

    for string in strings:
        digest = hash_ledger_entry(string, digest)
        yield digest, string

#### With this script, providing a set of strings will generate a unique and ordered ledger:

In [ ]:
for digest, string in generate_ledger("ninja","kongfu"):
    print(f"{digest}\t{string}")

#### These hash ledgers are tamper-resistant because the digests of later entries depend on the earlier entries. Modifying or adding entries will change the digest of later entries.

In [14]:
for digest, string in generate_ledger("ninja","kongfu","samurai"):
    print(f"{digest}\t{string}")

3899dcbab79f92af727c2190bbd8abc5	ninja
78d8c5d44aaad1bdcd8afe4e81181d82	kongfu
18b754a05052eebb96ca010c5361867c	samurai


#### We can also add a known ending entry to the ledger to protect the last entry from tampering:

In [ ]:
for digest, string in generate_ledger("ninja","pirate","samurai","END"):
    print(f"{digest}\t{string}")

### Validation
#### To validate a ledger, you can replay the transactions and make sure that you get the same hashes at each step:

In [21]:
our_digest = None
fileinput = open("BC.txt")
import sys

for line in fileinput.readlines():
    print(line)
    file_digest, word = line.strip().split("\t")
    our_digest = hash_ledger_entry(word, our_digest)

    if our_digest != file_digest:
        sys.exit(f"The digest for {word} does not match.")

print("All entries match.")
fileinput.close()

3899dcbab79f92af727c2190bbd8abc5	ninja

78d8c5d44aaad1bdcd8afe4e81181d82	kongfu

18b754a05052eebb96ca010c5361867c	samurai
All entries match.


In [15]:
digest

'18b754a05052eebb96ca010c5361867c'